In [69]:
# default_exp run_flow

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
# export


import os
import subprocess
from pathlib import Path

from nbdev.export import find_default_export, get_config, read_nb

from sciflow.utils import get_flow_path, prepare_env

# Verify and Run Sciflow Flows

In [73]:
nb_path = Path(os.path.join("test", "test_export.ipynb"))
flow_path = get_flow_path(nb_path, flow_provider="sagemaker")
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(get_config().path("lib_path"), f"{module_name}.py")
flows_dir = get_config(cfg_name="test/settings.ini").path("flows_path")
flow_name = os.path.basename(test_module)

In [55]:
# # export


# def prep_env():
#     if "USER" not in os.environ:
#         try:
#             os.environ["USER"] = os.environ["GIT_COMMITTER_NAME"]
#         except KeyError:
#             raise EnvironmentError(
#                 "Sciflow requires a known user for tracked execution. Add USER or GIT_COMMITTER_NAME to Jupyter environment variables"
#             )

In [74]:
# export


def run_shell_cmd(script: str):
    pipe = subprocess.Popen(
        "%s" % script, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True
    )
    output = pipe.communicate()[0]
    return pipe, output.decode("utf-8").strip()

In [82]:
# export


def check_call_flow(
    flow_nb_path, flow_provider="metaflow", flow_command="show", params=None
):
    prepare_env()
    if flow_nb_path.suffix == ".ipynb":
        flow_path = get_flow_path(nb_path, flow_provider=flow_provider)
    else:
        flow_path = flow_nb_path
    if params:
        args = " ".join([f"--{p[0]} {p[1]}" for p in params])
        flow_command = f"{flow_command} {args}"

    script = f"python '{flow_path}' {flow_command}"
    pipe, output = run_shell_cmd(script)
    return pipe.returncode, output

In [86]:
ret_code, output = check_call_flow(
    nb_path, flow_provider="metaflow", flow_command="show"
)
assert ret_code == 0

In [85]:
# slow

ret_code, output = check_call_flow(
    nb_path, flow_provider="metaflow", flow_command="run"
)
assert ret_code == 0
print(output)

Metaflow 2.5.2 executing TestExportFlow for user:'Donal Simmie'
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-05-05 08:33:51.607 Workflow starting (run-id 1651739631336278):
2022-05-05 08:33:51.672 [1651739631336278/start/1 (pid 28027)] Task is starting.
2022-05-05 08:33:56.415 [1651739631336278/start/1 (pid 28027)] 3
2022-05-05 08:33:56.749 [1651739631336278/start/1 (pid 28027)] Task finished successfully.
2022-05-05 08:33:56.846 [1651739631336278/preprocess/2 (pid 28030)] Task is starting.
2022-05-05 08:34:01.571 [1651739631336278/preprocess/2 (pid 28030)] Preprocessing input data from /home/sagemaker-user/git/sciflow/nbs...
2022-05-05 08:34:02.909 [1651739631336278/preprocess/2 (pid 28030)] Task finished successfully.
2022-05-05 08:34:02.994 [1651739631336278/train/3 (pid 28033)] Task is starting.
2022-05-05 08:34:07.713 [1651739631336278/train/3 (pid 28033)] Training /home/sagemaker-user/git/sciflow on /home/sagemaker-user/git/sciflow

In [88]:
ret_code, output = check_call_flow(
    nb_path, flow_provider="sagemaker", flow_command="show"
)
assert ret_code == 0

In [64]:
# slow

ret_code, output = check_call_flow(
    nb_path, flow_provider="sagemaker", flow_command="run"
)
assert ret_code == 0
print(output)

In [63]:
# export


def run_flow(
    nb_path, flow_provider="metaflow", flow_command="--no-pylint run", params=None
):
    flow_path = get_flow_path(nb_path, flow_provider=flow_provider)
    print(f"Running flow: {os.path.basename(flow_path)}")
    ret_code, output = check_flow(
        flow_path.parent,
        os.path.basename(flow_path),
        flow_provider=flow_provider,
        flow_command=flow_command,
        params=params,
    )
    return ret_code, output

In [37]:
# # export


# def check_flows(
#     config, flow_command="show", ignore_suffix="_no_params.py", exit_on_error=True
# ):
#     flow_results = {}
#     flows_dir = config.path("flows_path")
#     if ignore_suffix:
#         flow_paths = [p for p in os.listdir(flows_dir) if not p.endswith(ignore_suffix)]
#     else:
#         flow_paths = os.listdir(flows_dir)
#     ret_codes = []
#     exit_code = 0
#     for flow_path in flow_paths:
#         flow_name = os.path.basename(flow_path)
#         if flow_path.endswith(".py"):
#             ret_code, output = check_flow(
#                 flows_dir, flow_path, flow_command=flow_command
#             )
#             flow_results[flow_name] = ret_code, output
#             if ret_code == 0:
#                 print(f"Flow: {flow_name} {flow_command} verified")
#             else:
#                 print(
#                     f"Flow: {flow_name} {flow_command} verification failed\nDetails:\n{output}"
#                 )
#             ret_codes.append(ret_code)
#     if any([rc != 0 for rc in ret_codes]):
#         exit_code = 1
#         try:
#             # Exit with an error code if running from a non interactive Python environment.
#             get_ipython().__class__.__name__
#         except NameError:
#             if exit_on_error:
#                 return sys.exit(exit_code)
#     return exit_code

In [42]:
# check_flows(get_config(cfg_name="test/settings.ini"), ignore_suffix=None)

0

In [43]:
# slow

check_flows(get_config(cfg_name="test/settings.ini"), "--no-pylint run")

0

In [45]:
# # slow

# nb_path = os.path.join(Path(".").resolve(), "test", "test_multistep.ipynb")
# ret_code, output = run_flow(
#     nb_path,
#     params=[("traffic_percent", 1), ("model_level", "dispatcher")],
# )
# print(output)
# assert ret_code == 0

Running flow: test_multistep.py
Metaflow 2.5.2 executing TestMultistepFlow for user:'Donal Simmie'
Validating your flow...
    The graph looks good!
2022-04-15 14:49:34.787 Workflow starting (run-id 1650034174524893):
2022-04-15 14:49:34.848 [1650034174524893/start/1 (pid 22412)] Task is starting.
2022-04-15 14:49:36.262 [1650034174524893/start/1 (pid 22412)] The first step
2022-04-15 14:49:36.666 [1650034174524893/start/1 (pid 22412)] Task finished successfully.
2022-04-15 14:49:36.748 [1650034174524893/preprocess/2 (pid 22419)] Task is starting.
2022-04-15 14:49:38.170 [1650034174524893/preprocess/2 (pid 22419)] I captialised the message: THE FIRST STEP
2022-04-15 14:49:38.601 [1650034174524893/preprocess/2 (pid 22419)] Task finished successfully.
2022-04-15 14:49:38.683 [1650034174524893/fit/3 (pid 22426)] Task is starting.
2022-04-15 14:49:40.515 [1650034174524893/fit/3 (pid 22426)] Task finished successfully.
2022-04-15 14:49:40.593 [1650034174524893/evaluate/4 (pid 22433)] Task i

# Aynsc Flow Running

> Run the flow you are working on from the notebook you are working on. This maximises the amount of experiments you can run as you don't have down time. While long running tasks are running you can keep exploring! :-)

In [46]:
# # export


# async def run_flow_task(flow_path, param_grid=None):
#     flows_dir = flow_path.parent
#     flow_module = os.path.basename(flow_path)
#     flow_command = "--no-pylint run"
#     prep_mf_env()
#     if params:
#         args = " ".join([f"--{k} {v}" for k, v in param_grid.items()])
#         cmd = f"python '{os.path.join(flows_dir, flow_module)}' {flow_command} {args}"
#     else:
#         cmd = f"python '{os.path.join(flows_dir, flow_module)}' {flow_command}"
#     proc = await asyncio.create_subprocess_shell(
#         cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
#     )

#     stdout, stderr = await proc.communicate()

#     print(f"[{cmd!r} exited with {proc.returncode}]")
#     if stdout:
#         output = stdout.decode("utf-8").strip()
#         print(f"[stdout]\n{output}")
#     if stderr:
#         print(f'[stderr]\n{stderr.decode("utf-8").strip()}')

#     return proc.returncode

In [47]:
# # export


# def run_flow_async(nb_path, params=None):
#     flow_path = get_flow_path(nb_path)
#     loop = asyncio.get_event_loop()
#     task = loop.create_task(run_flow_task(flow_path, params))
#     return task

In [48]:
# # slow

# task = run_flow_async(
#     os.path.join("test", "test_multistep.ipynb"),
#     params={"traffic_percent": 10, "workers": 12},
# )
# task

<Task pending name='Task-4' coro=<run_flow_task() running at /tmp/ipykernel_22384/1238802591.py:4>>

In [49]:
# # slow
# await task

["python '/home/sagemaker-user/git/sciflow/nbs/test/flows/metaflow/test_multistep.py' --no-pylint run --traffic_percent 10 --workers 12" exited with 0]
[stdout]
2022-04-15 14:49:46.509 Workflow starting (run-id 1650034186248371):
2022-04-15 14:49:46.573 [1650034186248371/start/1 (pid 22456)] Task is starting.
2022-04-15 14:49:47.983 [1650034186248371/start/1 (pid 22456)] The first step
2022-04-15 14:49:48.378 [1650034186248371/start/1 (pid 22456)] Task finished successfully.
2022-04-15 14:49:48.463 [1650034186248371/preprocess/2 (pid 22463)] Task is starting.
2022-04-15 14:49:49.887 [1650034186248371/preprocess/2 (pid 22463)] I captialised the message: THE FIRST STEP
2022-04-15 14:49:50.324 [1650034186248371/preprocess/2 (pid 22463)] Task finished successfully.
2022-04-15 14:49:50.405 [1650034186248371/fit/3 (pid 22470)] Task is starting.
2022-04-15 14:49:52.237 [1650034186248371/fit/3 (pid 22470)] Task finished successfully.
2022-04-15 14:49:52.317 [1650034186248371/evaluate/4 (pid 22

0

In [50]:
# param_grid = {
#     "traffic_percent": [1, 5, 10, 20, 50, 100],
#     "model_level": ["router", "dispatcher"],
#     "workers": [1],
# }

In [51]:
# # export


# def iter_param_grid(param_grid):
#     # https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/model_selection/_search.py
#     for p in [param_grid]:
#         # Always sort the keys of a dictionary, for reproducibility
#         items = sorted(p.items())
#         if not items:
#             yield {}
#         else:
#             keys, values = zip(*items)
#             for v in product(*values):
#                 params = dict(zip(keys, v))
#                 yield params

In [52]:
# assert [{"a": 1, "b": 1, "c": "hello"}, {"a": 2, "b": 1, "c": "hello"}] == list(
#     iter_param_grid({"a": [1, 2], "b": [1], "c": ["hello"]})
# )

In [53]:
# # export


# def sample_grid_space(param_grid: Dict[str, Iterable[Any]], num_samples: int):
#     samples = []
#     for i, sample in enumerate(iter_param_grid(param_grid)):
#         samples.append(sample)
#     if num_samples < len(samples):
#         samples = pd.Series(samples).sample(num_samples).tolist()
#     return samples

In [54]:
# sample_space = sample_grid_space({"a": [1, 2], "b": [1], "c": ["hello"]}, 1)
# assert sample_space[0]["b"] == 1
# assert sample_space[0]["c"] == "hello"
# assert sample_space[0]["a"] == 1 or sample_space[0]["a"] == 2

In [55]:
# # export


# def search_flow_grid(nb_path, param_grid, num_procs=None):
#     if num_procs is None:
#         num_procs = int((multiprocessing.cpu_count() / 2) - 1)

#     param_sample_space = sample_grid_space(param_grid, num_procs)
#     tasks = []
#     for param_sample in param_sample_space:
#         tasks.append(run_flow_async(nb_path, params=param_sample))
#     return tasks

In [56]:
# # slow
# nb_path = Path(
#     os.path.join(
#         Path(".").resolve(),
#         "test",
#         "test_multistep.ipynb",
#     )
# )
# tasks = search_flow_grid(
#     nb_path,
#     {
#         "traffic_percent": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50],
#         "model_level": ["dispatcher"],
#         "workers": [1],
#     },
#     2,
# )

In [59]:
# # slow

# [t.done() for t in tasks]

[True, True]

In [60]:
# # slow

# [t.result() for t in tasks]

[0, 0]

# Folding @ Home Type Exploration

> Explore wider search space in background. Try to always be making some use of resource. Needs persistent search space tracking.

In [61]:
# # export


# @call_parse
# def sciflow_check_flows():
#     check_flows(get_config())

In [62]:
# # export


# @call_parse
# def sciflow_run_flows():
#     check_flows(get_config(), "--no-pylint run")